In [1]:
import json
from tqdm import trange
import random
from collections import Counter
random.seed(25000)

In [2]:
def gen(step):
    var='num_0'
    code=[]
    for i in range(step):
        var=f'num{i}_'
        op=random.choice('+-')
        if i==0:
            num1=random.randint(0,22)
            num2=random.randint(0,22)
        else:
            if random.randint(0,1):
                num1=random.choice(list(range(23)))
                num2=f'num{i-1}_'
            else:
                num1=f'num{i-1}_'
                num2=random.choice(list(range(23)))
        code.append(var+'='+str(num1)+op+str(num2))
    exe_code='\n'.join(code)
    exec(exe_code)
    return code,eval(var)

In [ ]:
data_dup=[]
template=[]

for i in range(0,23):
    for j in range(0,23):
        tar=''
        template.append({"premise": f'num0_={str(i)}+{str(j)}', 'target':'num0_', 'gt': str((i+j)%23), 'step':str(1)})
        template.append({"premise": f'num0_={str(i)}-{str(j)}', 'target':'num0_', 'gt': str((i-j)%23), 'step':str(1)})
print(len(template))

for step in trange(1,5):
    data_cnt=25000
    while(data_cnt>0):
        gen_code,val=gen(step+1)
        eqs = '\n'.join(gen_code)  
        if eqs in data_dup:
            continue
        template.append({"premise": eqs, 'target':f'num{step}_', 'gt': str(val%23), 'step':str(step+1)})
        data_cnt-=1
        data_dup.append(eqs)
    print(len(template))
print(random.choice(template))


1058


 25%|██▌       | 1/4 [00:04<00:14,  4.79s/it]

26058


 50%|█████     | 2/4 [00:13<00:14,  7.39s/it]

51058


 75%|███████▌  | 3/4 [00:28<00:10, 10.47s/it]

76058


100%|██████████| 4/4 [00:47<00:00, 11.82s/it]

101058
{'premise': 'num0_=18-12\nnum1_=21-num0_\nnum2_=num1_-14\nnum3_=num2_-1\nnum4_=num3_-14', 'target': 'num4_', 'gt': 9, 'step': '5'}


In [ ]:
test_template=[]
for step in trange(1,7):
    data_cnt=500
    while(data_cnt>0):
        gen_code,val=gen(step+1)
        dup=False
        for i in range(2,len(gen_code)):
            if '\n'.join(gen_code[:i]) in data_dup:
                dup=True
        if dup==True:
            continue
        eqs = '\n'.join(gen_code)  
        test_template.append({"premise": eqs, 'target':f'num{step}_', 'gt': str(val%23), 'step':str(step+1)})
        data_cnt-=1
        data_dup.append('\n'.join(gen_code))
    print(len(test_template))
print(random.choice(test_template))

  0%|          | 0/6 [00:00<?, ?it/s]

500


 33%|███▎      | 2/6 [00:00<00:01,  3.87it/s]

1000


 50%|█████     | 3/6 [00:01<00:01,  1.62it/s]

...

2000


 83%|████████▎ | 5/6 [00:05<00:01,  1.50s/it]

2500


100%|██████████| 6/6 [00:08<00:00,  1.43s/it]

3000
{'premise': 'num0_=13+10\nnum1_=num0_+19\nnum2_=num1_-1\nnum3_=4+num2_', 'target': 'num3_', 'gt': 22, 'step': '4'}


In [5]:
def replace(input_text,num_list):
    for i in range(26):
        input_text = input_text.replace(f'num{i}_', num_list[i])
    return input_text

In [6]:
random.seed(1)
random.shuffle(template)

train_dataset=[]
test_dataset=[]

random.seed(11)
for item in template[:-400]:
    alpha_list=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    train_dataset.append({'template':item['premise'], 'premise': replace(item['premise'], alpha_list), 'target':alpha_list[int(item['step'])-1], 'gt': item['gt'], 'step':item['step'], 'type':'train'})
    alpha_list2=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    while(alpha_list==alpha_list2):
        alpha_list2=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    train_dataset.append({'template':item['premise'], 'premise': replace(item['premise'], alpha_list2), 'target':alpha_list2[int(item['step'])-1], 'gt': item['gt'], 'step':item['step'], 'type':'train'})
random.seed(21)
for item in template[-400:]:
    alpha_list=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    train_dataset.append({'template':item['premise'], 'premise': replace(item['premise'], alpha_list), 'target':alpha_list[int(item['step'])-1], 'gt': item['gt'], 'step':item['step'], 'type':'train'})
    alpha_list2=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    while(alpha_list==alpha_list2):
        alpha_list2=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    test_dataset.append({'template':item['premise'], 'premise': replace(item['premise'], alpha_list2), 'target':alpha_list2[int(item['step'])-1], 'gt': item['gt'], 'step':item['step'], 'type':'test_same'})
random.seed(31)
for item in test_template:
    alpha_list=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    test_dataset.append({'template':item['premise'], 'premise': replace(item['premise'], alpha_list), 'target':alpha_list[int(item['step'])-1], 'gt': item['gt'], 'step':item['step'], 'type':'test'})
    
random.seed(1)
test_dataset+=random.sample(train_dataset,400)
Counter([item['type'] for item in test_dataset])


Counter({'test': 3000, 'test_same': 400, 'train': 400})

In [7]:
random.choice(train_dataset)

{'template': 'num0_=5+19\nnum1_=num0_-14',
 'premise': 'f=5+19\nn=f-14',
 'target': 'n',
 'gt': 10,
 'step': '2',
 'type': 'train'}

In [8]:
random.choice(test_dataset)

{'template': 'num0_=15-3\nnum1_=num0_-10\nnum2_=num1_-4\nnum3_=num2_+22\nnum4_=num3_-10\nnum5_=22+num4_\nnum6_=1-num5_',
 'premise': 'g=15-3\np=g-10\nh=p-4\nx=h+22\nj=x-10\nl=22+j\nd=1-l',
 'target': 'd',
 'gt': 15,
 'step': '7',
 'type': 'test'}

In [9]:
random.seed(1)
random.shuffle(train_dataset)

with open('train.json','w') as f:
    print(len(train_dataset))
    json.dump(train_dataset,f)
with open('test.json','w') as f:
    print(len(test_dataset))
    json.dump(test_dataset,f)

201716
3800


# Generate variable as subtrahend

In [ ]:
import re

data = """
b=10-1
k=20-b
r=20+k
p=17-r
d=p-15"""
def get_num(data):
    pattern = r'\d+-[a-zA-Z]'
    lines = data.strip().split('\n')
    count = sum(1 for line in lines if re.search(pattern, line))
    return count
get_num(data)

2

In [ ]:
step=5

from tqdm import tqdm
vas_template=[]
count_dict = {}  
for item in range(step):
    count_dict[item]=0
for item in test_template:
    if item['step']==str(step) and count_dict[get_num(item['premise'])]<100:
        item['var_as_sub']=get_num(item['premise'])
        vas_template.append(item)
        count_dict[get_num(item['premise'])]+=1

progress_bar = tqdm(total=step*100-sum(count_dict.values()), desc="Generating Data")  
while any(count < 100 for count in count_dict.values()):  
    gen_code,val=gen(step)
    dup=False
    for i in range(2,len(gen_code)):
        if '\n'.join(gen_code[:i]) in data_dup:
            dup=True
    if dup==True:
        continue
    eqs = '\n'.join(gen_code)  
    count_value = get_num(eqs)  
    if count_value in count_dict and count_dict[count_value] < 100:  
        vas_template.append({"premise": eqs, 'target':f'num{step}_', 'gt': str(val%23), 'step':str(step), 'var_as_sub':count_value})
        count_dict[count_value] += 1  
        progress_bar.update(1)  
progress_bar.close()
Counter([get_num(item['premise']) for item in vas_template])


Generating Data:   0%|          | 0/171 [00:00<?, ?it/s]

Generating Data: 100%|██████████| 171/171 [01:40<00:00,  1.69it/s]


Counter({2: 100, 0: 100, 3: 100, 1: 100, 4: 100})

In [30]:
random.seed(31)
vas_dataset=[]
for item in vas_template:
    alpha_list=random.sample(list('abcdefghijklmnopqrstuvwxyz'),26)
    vas_dataset.append({'template':item['premise'], 'premise': replace(item['premise'], alpha_list), 'target':alpha_list[int(item['step'])-1], 'gt': item['gt'], 'step':item['step'], 'var_as_sub':item['var_as_sub'], 'type':'test'})
with open('test_vas.json','w') as f:
    print(len(vas_dataset))
    json.dump(vas_dataset,f)

500
